## Deploy Azure ML Models to IoT Edge Devices - Using Python SDK
You can use the Azure Machine Learning Python SDK to deploy models to IoT Edge devices if your IoT Hub is in the same subscription as your Azure ML Workspace. Otherwise, you will need to deploy using the Azure IoT CLI or the Azure Portal. 


### 1. Retrieve your Azure ML Workspace

In [7]:
from azureml.core import Workspace
ws = Workspace.from_config()
iot_hub_name = "fermi-iot"
iot_device_id = "fermi-edge"

### 3. Attach your IoT Hub as Azure ML Compute


In [9]:
from azureml.contrib.core.compute import IotHubCompute
iothub_compute = IotHubCompute(ws, iot_device_id)

List Azure ML Compute Targets.

In [10]:
from azureml.core.compute import ComputeTarget

compute_targets = ComputeTarget.list(ws)
for t in compute_targets: 
    if t.type == "IotHub":
        print("IotHub '{}' has provisioning state '{}'.".format(t.name, t.provisioning_state))

IotHub 'fermi-edge' has provisioning state 'Succeeded'.


### 4. Configure Azure ML module

In [11]:
from azureml.contrib.core.webservice import IotWebservice, IotBaseModuleSettings, IotModuleSettings

#Pick a module name
module_name = "test-module-4"

In [12]:
container_config = '{ \
  "HostConfig": { \
    "Binds": [ \
      "/etc/hosts:/etc/hosts" \
    ], \
    "Privileged": true, \
    "Devices": [ \
      { \
        "PathOnHost": "/dev/catapult0", \
        "PathInContainer": "/dev/catapult0" \
      }, \
      { \
        "PathOnHost": "/dev/catapult1", \
        "PathInContainer": "/dev/catapult1" \
      } \
    ], \
    "PortBindings": { \
      "50051/tcp": [ \
        { \
          "HostPort": "50051" \
        } \
      ] \
    } \
  } \
}'

routes = {
    "route": "FROM /messages/* INTO "
}

# Here, we define the Azure ML module with the container_config options above
aml_module = IotBaseModuleSettings(name=module_name, create_option=container_config)

#### b. Deploy from ContainerImage

If you've already created an AccelContainerImage with the converted model that you want to run, then you can use the cell below to pull that image and deploy it. This image is stored in your Azure Container Registry (ACR) associated with your Azure ML Workspace.

In [13]:
from azureml.core import Image, Webservice
from azureml.contrib.core.webservice import IotWebservice
from azureml.accel import AccelContainerImage

# This time, we will leave off the external module from the deployment manifest
deploy_config = IotWebservice.deploy_configuration(device_id=iot_device_id, routes=routes, aml_module=aml_module)

# Deploy from latest version of image, using module_name as your IotWebservice name
image_name = "im-klijnsma-tquarkrn50-v1-s0"
iot_service_name = module_name

# Can specify version=x, otherwise will grab latest
image = Image(ws, image_name) 
iot_service = IotWebservice.deploy_from_image(ws, iot_service_name, image, deploy_config, iothub_compute)
iot_service.wait_for_deployment()

IOT service creation operation finished, operation "Succeeded"
